In [1]:
from math import floor, log
from collections import defaultdict
import random
import tqdm
import numpy as np
from numpy.random import poisson, binomial, uniform, exponential, multinomial, normal, multivariate_normal
from scipy.stats import norm
from scipy.stats import kstest, norm

def dice():
    return random.randint(1, 6)
ct = 10000

def multi(dist):
    v = multinomial(1, dist)
    v = np.where(v == 1)[0][0]
    return v

def markov_start(start, P, length):
    '''
    Runs Markov Chain with a start value, generates {length} items. 
    '''
    cur = start
    seq = []
    for i in range(length):
        distribution = P[cur].flatten()
        v = multinomial(1, distribution)
        v = np.where(v == 1)[0][0]
        seq.append(v)
        cur = v
    return seq

def markov_start_absorb(start, P, absorb_set):
    '''
    Runs Markov Chain with a start value, generates {length} items. 
    '''
    cur = start
    seq = []
    while True:
        distribution = P[cur].flatten()
        v = multi(distribution)
        seq.append(v)
        cur = v
        if cur in absorb_set:
            break
    return seq

def markov_rdm(dist, P, length):
    '''
    Runs Markov Chain with a starting distribution, generates {length} + 1 items. 
    '''
    cur = np.nonzero(multinomial(1, dist))[0]
    seq = [cur]
    for i in range(length):
        distribution = P[cur].flatten()
        v = np.nonzero(multinomial(1, distribution))[0]
        seq.append(v)
        cur = v
    return seq

In [6]:
def q1():
    succ = 0
    for i in tqdm.tqdm(range(50000)):
        li = [0, random.randint(0, 1)]
        while li[-1] != li[-2]:
            li.append(random.randint(0, 1))
        if li[-2:] == [1, 1]:
            succ += 1
    print(f"Empirical: {succ / 50000}")
q1()

100%|██████████| 50000/50000 [00:00<00:00, 399007.60it/s]

Empirical: 0.33118


In [8]:
def qi():
    ct = 0
    for i in tqdm.tqdm(range(50000)):
        start = 1
        cur = random.choice([0, 2])
        steps = 1
        while cur != start:
            cur = random.choice([(cur - 1) % 12, (cur + 1) % 12])
            steps += 1
        ct += steps
    print(f"Empirical: {ct / 50000}")
qi() 


100%|██████████| 50000/50000 [00:00<00:00, 107526.96it/s]

Empirical: 11.94528


In [12]:
def qii():
    succ = 0
    for i in tqdm.tqdm(range(500000)):
        start = 1
        cur = random.choice([0, 2])
        visited = set([cur])
        while not (len(visited) < 12 and cur == 1 or len(visited) == 12):
            cur = random.choice([(cur - 1) % 12, (cur + 1) % 12])
            visited.add(cur)
        if len(visited) == 12:
            succ += 1
    print(f"Empirical: {succ / 500000}")
    print(f"Analytical: {1 / 11}")
qii()


100%|██████████| 500000/500000 [00:05<00:00, 89670.03it/s]

Empirical: 0.091284
Analytical: 0.09090909090909091


In [4]:
def q3():
    N = 50
    totals = 0
    for i in tqdm.tqdm(range(20000)):
        collected = set()
        iters = 0
        while len(collected) < N:
            collected.add(random.randint(1, N))
            iters += 1
        totals += iters
    print(f"Empirical: {totals / 20000}")
    print(f"1/2 (1 + N) {0.5 * (1 + N)}")
    print(f"N * sum(1/1 + .. + 1/N) {N * sum([1/i for i in range(1, N + 1)])}")
q3()
        

100%|██████████| 20000/20000 [00:04<00:00, 4620.00it/s]

Empirical: 225.0518
1/2 (1 + N) 25.5
N * sum(1/1 + .. + 1/N) 224.96026691647114


In [27]:
from collections import defaultdict
def q4():
    dic = defaultdict(int)
    a, b = 0.5, 0.5
    P = np.array([[1-a, a], [b, 1-b]])
    for i in tqdm.tqdm(range(10000)):
        chain = markov_start_absorb(0, P, set([0]))
        dic[len(chain)] += 1
    print(f"alpha: {a}, beta: {b}")
    print("Empirical: ", [(k, v / 10000) for k, v in sorted(dic.items())])
    print("Expected: ", [(k, a * ((1 - b) **(k - 2)) * b if k != 1 else 1 - a) for k, v in sorted(dic.items())])
q4()

    

100%|██████████| 10000/10000 [00:00<00:00, 31545.84it/s]

alpha: 0.5, beta: 0.5
Empirical:  [(1, 0.5021), (2, 0.2464), (3, 0.1248), (4, 0.0657), (5, 0.0297), (6, 0.0159), (7, 0.0071), (8, 0.0042), (9, 0.002), (10, 0.001), (11, 0.0005), (12, 0.0002), (13, 0.0002), (14, 0.0001), (16, 0.0001)]
Expected:  [(1, 0.5), (2, 0.25), (3, 0.125), (4, 0.0625), (5, 0.03125), (6, 0.015625), (7, 0.0078125), (8, 0.00390625), (9, 0.001953125), (10, 0.0009765625), (11, 0.00048828125), (12, 0.000244140625), (13, 0.0001220703125), (14, 6.103515625e-05), (16, 1.52587890625e-05)]


In [40]:
def q5():
    g = np.array([
        [1, 0, 1, 0, 0, 0], 
        [0, 1, 0, 1, 0, 0], 
        [1, 0, 1, 0, 0, 0], 
        [0, 1, 0, 1, 0, 0], 
        [1, 1, 0, 0, 1, 1],
        [1, 1, 1, 1, 1, 1]])
    rev = g.T
    topo_order = []
    visited = set()
    def clock_explore(node):
        if node not in visited:
            visited.add(node)
            for i, v in enumerate(rev[node]):
                if v == 1:
                    clock_explore(i)
            topo_order.append(node)
    for i in range(6):
        clock_explore(i)
    rev_top = reversed(topo_order)

    visited = set()
    scc = []
    def scc_dfs(node):
        if node not in visited:
            visited.add(node)
            scc[-1].append(node)
            for i, v in enumerate(g[node]):
                if v == 1:
                    scc_dfs(i)

    for i in rev_top:
        if i not in visited:
            scc.append([])
            scc_dfs(i)
    print(scc)
q5()

[[1, 3], [0, 2], [5, 4]]
